In [1]:
!pip install numpy
!pip install matplotlib
!pip install --upgrade pip setuptools wheel
!pip install opencv-python
# !apt-get update
# !apt-get install ffmpeg libsm6 libxext6  -y
import numpy as np
import json
import matplotlib.pyplot as plt
import os
import cv2
import datetime
import dateutil.relativedelta
import math

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
allFiles = os.listdir('/home/team19/Desktop/Axis_DL/Detection/YOLO/05-06-2021-21:17:32/Camera 1/')
jsonList = []
files = []
for f in allFiles:
        if ".jpg" in f:
            if "._" not in f:
                files.append(f)
        if '.json' in f:
            if '._' not in f:
                jsonList.append(f)
ts = datetime.datetime.fromtimestamp(float(jsonList[0][:-5]))
len(jsonList)


46794

In [9]:
def draw_xywha(im, num, x, y, w, h, angle, color=(0,255,0), linewidth=5):
    '''
    im: image numpy array, shape(h,w,3), RGB
    angle: degree
    '''
    font = cv2.FONT_HERSHEY_SIMPLEX
    c, s = np.cos(angle/180*np.pi), np.sin(angle/180*np.pi)
    R = np.asarray([[c, s], [-s, c]])
    pts = np.asarray([[-w, -h], [w, -h], [w, h], [-w, h]])
    rot_pts = []
    for pt in pts:
        rot_pts.append(([2*x, 2*y] + pt @ R).astype(int))
    contours = np.array([rot_pts[0], rot_pts[1], rot_pts[2], rot_pts[3]])
    cv2.polylines(im, [contours], isClosed=True, color=color,
                thickness=linewidth, lineType=cv2.LINE_4)
    cv2.putText(im,str(num),(int(2 *x)-15, int(2 *y)), font, 2, (0, 255, 0), 3, cv2.LINE_AA)

def drawing_bb(folder_path, video_name, RP1, RP2):

    out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v') ,5, (1024, 1024))
    allFiles = os.listdir(folder_path)
    jsonList = []
    files = []
    RP = RP1 + RP2
    RP = sorted(RP)
    RP_index = 0
    RP = [f for f in RP if '._' not in f]
    print(RP)
    for f in allFiles:
        if ".jpg" in f:
            if "._" not in f:
                files.append(f)
        if '.json' in f:
            if '._' not in f:
                jsonList.append(f)
#     print(files)
    jsonList = sorted(jsonList)
    files = sorted(files)
#     print(len(jsonList), len(files))
    jsonList = jsonList[40000:]
    files = files[40000:]
    RP1count = 0
    RP2count = 0
    for i in range(len(files)):
        loc =  folder_path + files[i]
        jloc = folder_path + jsonList[i]
        with open(jloc) as f:
            cam1 = json.load(f)
        font = cv2.FONT_HERSHEY_SIMPLEX
        image = cv2.imread(loc)
        boxes = list(cam1["Bounding Box"].keys())
        count = 0
        for b in boxes:
            count+=1
            draw_xywha(image, count, cam1["Bounding Box"][b][0], cam1["Bounding Box"][b][1], cam1["Bounding Box"][b][2], cam1["Bounding Box"][b][3], cam1["Bounding Box"][b][4], color=(0,255,0), linewidth=5)
#             cv2.putText(image,str(count),(int(cam1["Bounding Box"][b][0])-15, int(cam1["Bounding Box"][b][1])), font, 2, (255, 0, 0), 3, cv2.LINE_AA)
        cv2.putText(image,"People#: " + str(cam1["Num People"]),(1600, 50), font, 2, (0, 0, 255), 3, cv2.LINE_AA)
        ts = datetime.datetime.fromtimestamp(float(jsonList[0][:-5]))
        ts2 = datetime.datetime.fromtimestamp(float(cam1['Timestamp']))
        diff = dateutil.relativedelta.relativedelta(ts2, ts)
        print(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)/60), (diff.hours*3600 + diff.minutes * 60 + diff.seconds)%60)    
        cv2.putText(image,"since: " + str(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)/60))+"min "+str(math.floor((diff.hours*3600 + diff.minutes * 60 + diff.seconds)%60))+"sec",(1600, 120), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        
        RP_relative = float(RP[RP_index].replace('.json', ''))
        if float(cam1['Timestamp']) > RP_relative:
            if RP[RP_index] in RP1:
                with open('/home/team19/COSSY/RP1/' + RP[RP_index]) as f:
                    RPinfo = json.load(f)
                RP1count = int(RPinfo['count'])
            elif RP[RP_index] in RP2:
                with open('/home/team19/COSSY/RP2/' + RP[RP_index]) as f:
                    RPinfo = json.load(f)
                RP2count = int(RPinfo['count'])
            if RP_index < len(RP)-1:
                RP_index += 1
        print("TDS1: " + str(RP1count), "TDS2: " + str(RP2count))
        cv2.putText(image, "TDS1: " + str(RP2count),(1700, 1900), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        cv2.putText(image, "TDS2: " + str(RP1count),(1700, 1950), font, 1.3, (0, 0, 255), 3, cv2.LINE_AA)
        resized = cv2.resize(image, (1024,1024))
        out.write(resized)
        
#     for data_file in sorted(files):
#         loc = folder_path+data_file
#         print(loc)
#         image = cv2.imread(loc)
        
#         resized = cv2.resize(image, (1024,1024))

#         out.write(resized)

        
    out.release()

In [10]:
folder_path = '/home/team19/Desktop/Axis_DL/Detection/YOLO/05-06-2021-21:17:32/Camera 1/'
video_name = 'trial5.mp4'
RP1 = os.listdir('/home/team19/COSSY/RP1/')
RP2 = os.listdir('/home/team19/COSSY/RP2/')
drawing_bb(folder_path, video_name, RP1, RP2)

['1620362781.39.json', '1620362931.547.json', '1620365440.663.json', '1620368144.517.json', '1620368154.981.json', '1620377429.204.json', '1620378123.686.json', '1620381060.455.json', '1620387996.631.json']
0 0
TDS1: 1 TDS2: 0
0 2
TDS1: 0 TDS2: 0
0 4
TDS1: 0 TDS2: 1
0 6
TDS1: 0 TDS2: 0
0 8
TDS1: 0 TDS2: -1
0 10
TDS1: 0 TDS2: -2
0 12
TDS1: 0 TDS2: -1
0 14
TDS1: 0 TDS2: -2
0 16
TDS1: 1 TDS2: -2
0 18
TDS1: 1 TDS2: -2
0 20
TDS1: 1 TDS2: -2
0 22
TDS1: 1 TDS2: -2
0 24
TDS1: 1 TDS2: -2
0 26
TDS1: 1 TDS2: -2
0 28
TDS1: 1 TDS2: -2
0 30
TDS1: 1 TDS2: -2
0 32
TDS1: 1 TDS2: -2
0 34
TDS1: 1 TDS2: -2
0 36
TDS1: 1 TDS2: -2
0 38
TDS1: 1 TDS2: -2
0 40
TDS1: 1 TDS2: -2
0 43
TDS1: 1 TDS2: -2
0 44
TDS1: 1 TDS2: -2
0 47
TDS1: 1 TDS2: -2
0 49
TDS1: 1 TDS2: -2
0 51
TDS1: 1 TDS2: -2
0 53
TDS1: 1 TDS2: -2
0 55
TDS1: 1 TDS2: -2
0 57
TDS1: 1 TDS2: -2
0 59
TDS1: 1 TDS2: -2
1 1
TDS1: 1 TDS2: -2
1 3
TDS1: 1 TDS2: -2
1 5
TDS1: 1 TDS2: -2
1 7
TDS1: 1 TDS2: -2
1 9
TDS1: 1 TDS2: -2
1 11
TDS1: 1 TDS2: -2
1 13
TDS1: 1 TD

12 23
TDS1: 1 TDS2: -2
12 25
TDS1: 1 TDS2: -2
12 27
TDS1: 1 TDS2: -2
12 29
TDS1: 1 TDS2: -2
12 31
TDS1: 1 TDS2: -2
12 33
TDS1: 1 TDS2: -2
12 35
TDS1: 1 TDS2: -2
12 37
TDS1: 1 TDS2: -2
12 39
TDS1: 1 TDS2: -2
12 41
TDS1: 1 TDS2: -2
12 43
TDS1: 1 TDS2: -2
12 45
TDS1: 1 TDS2: -2
12 47
TDS1: 1 TDS2: -2
12 49
TDS1: 1 TDS2: -2
12 51
TDS1: 1 TDS2: -2
12 53
TDS1: 1 TDS2: -2
12 56
TDS1: 1 TDS2: -2
12 58
TDS1: 1 TDS2: -2
13 0
TDS1: 1 TDS2: -2
13 2
TDS1: 1 TDS2: -2
13 4
TDS1: 1 TDS2: -2
13 6
TDS1: 1 TDS2: -2
13 8
TDS1: 1 TDS2: -2
13 10
TDS1: 1 TDS2: -2
13 12
TDS1: 1 TDS2: -2
13 14
TDS1: 1 TDS2: -2
13 16
TDS1: 1 TDS2: -2
13 18
TDS1: 1 TDS2: -2
13 20
TDS1: 1 TDS2: -2
13 22
TDS1: 1 TDS2: -2
13 24
TDS1: 1 TDS2: -2
13 26
TDS1: 1 TDS2: -2
13 28
TDS1: 1 TDS2: -2
13 30
TDS1: 1 TDS2: -2
13 32
TDS1: 1 TDS2: -2
13 34
TDS1: 1 TDS2: -2
13 37
TDS1: 1 TDS2: -2
13 39
TDS1: 1 TDS2: -2
13 41
TDS1: 1 TDS2: -2
13 43
TDS1: 1 TDS2: -2
13 45
TDS1: 1 TDS2: -2
13 47
TDS1: 1 TDS2: -2
13 49
TDS1: 1 TDS2: -2
13 51
TDS1: 1 TD

24 40
TDS1: 1 TDS2: -2
24 42
TDS1: 1 TDS2: -2
24 44
TDS1: 1 TDS2: -2
24 46
TDS1: 1 TDS2: -2
24 48
TDS1: 1 TDS2: -2
24 50
TDS1: 1 TDS2: -2
24 52
TDS1: 1 TDS2: -2
24 54
TDS1: 1 TDS2: -2
24 56
TDS1: 1 TDS2: -2
24 58
TDS1: 1 TDS2: -2
25 0
TDS1: 1 TDS2: -2
25 2
TDS1: 1 TDS2: -2
25 4
TDS1: 1 TDS2: -2
25 6
TDS1: 1 TDS2: -2
25 8
TDS1: 1 TDS2: -2
25 10
TDS1: 1 TDS2: -2
25 13
TDS1: 1 TDS2: -2
25 15
TDS1: 1 TDS2: -2
25 17
TDS1: 1 TDS2: -2
25 19
TDS1: 1 TDS2: -2
25 21
TDS1: 1 TDS2: -2
25 23
TDS1: 1 TDS2: -2
25 25
TDS1: 1 TDS2: -2
25 27
TDS1: 1 TDS2: -2
25 29
TDS1: 1 TDS2: -2
25 31
TDS1: 1 TDS2: -2
25 33
TDS1: 1 TDS2: -2
25 35
TDS1: 1 TDS2: -2
25 37
TDS1: 1 TDS2: -2
25 39
TDS1: 1 TDS2: -2
25 41
TDS1: 1 TDS2: -2
25 43
TDS1: 1 TDS2: -2
25 45
TDS1: 1 TDS2: -2
25 47
TDS1: 1 TDS2: -2
25 49
TDS1: 1 TDS2: -2
25 51
TDS1: 1 TDS2: -2
25 53
TDS1: 1 TDS2: -2
25 55
TDS1: 1 TDS2: -2
25 57
TDS1: 1 TDS2: -2
25 59
TDS1: 1 TDS2: -2
26 1
TDS1: 1 TDS2: -2
26 4
TDS1: 1 TDS2: -2
26 6
TDS1: 1 TDS2: -2
26 8
TDS1: 1 TDS2: 

36 58
TDS1: 1 TDS2: -2
37 0
TDS1: 1 TDS2: -2
37 2
TDS1: 1 TDS2: -2
37 4
TDS1: 1 TDS2: -2
37 6
TDS1: 1 TDS2: -2
37 8
TDS1: 1 TDS2: -2
37 10
TDS1: 1 TDS2: -2
37 12
TDS1: 1 TDS2: -2
37 14
TDS1: 1 TDS2: -2
37 16
TDS1: 1 TDS2: -2
37 19
TDS1: 1 TDS2: -2
37 21
TDS1: 1 TDS2: -2
37 23
TDS1: 1 TDS2: -2
37 25
TDS1: 1 TDS2: -2
37 27
TDS1: 1 TDS2: -2
37 29
TDS1: 1 TDS2: -2
37 31
TDS1: 1 TDS2: -2
37 33
TDS1: 1 TDS2: -2
37 35
TDS1: 1 TDS2: -2
37 37
TDS1: 1 TDS2: -2
37 39
TDS1: 1 TDS2: -2
37 41
TDS1: 1 TDS2: -2
37 43
TDS1: 1 TDS2: -2
37 45
TDS1: 1 TDS2: -2
37 47
TDS1: 1 TDS2: -2
37 49
TDS1: 1 TDS2: -2
37 51
TDS1: 1 TDS2: -2
37 53
TDS1: 1 TDS2: -2
37 55
TDS1: 1 TDS2: -2
37 57
TDS1: 1 TDS2: -2
38 0
TDS1: 1 TDS2: -2
38 2
TDS1: 1 TDS2: -2
38 4
TDS1: 1 TDS2: -2
38 6
TDS1: 1 TDS2: -2
38 8
TDS1: 1 TDS2: -2
38 10
TDS1: 1 TDS2: -2
38 12
TDS1: 1 TDS2: -2
38 14
TDS1: 1 TDS2: -2
38 16
TDS1: 1 TDS2: -2
38 18
TDS1: 1 TDS2: -2
38 20
TDS1: 1 TDS2: -2
38 22
TDS1: 1 TDS2: -2
38 24
TDS1: 1 TDS2: -2
38 26
TDS1: 1 TDS2: -

49 16
TDS1: 1 TDS2: -2
49 18
TDS1: 1 TDS2: -2
49 20
TDS1: 1 TDS2: -2
49 22
TDS1: 1 TDS2: -2
49 24
TDS1: 1 TDS2: -2
49 26
TDS1: 1 TDS2: -2
49 28
TDS1: 1 TDS2: -2
49 30
TDS1: 1 TDS2: -2
49 32
TDS1: 1 TDS2: -2
49 35
TDS1: 1 TDS2: -2
49 37
TDS1: 1 TDS2: -2
49 39
TDS1: 1 TDS2: -2
49 41
TDS1: 1 TDS2: -2
49 43
TDS1: 1 TDS2: -2
49 45
TDS1: 1 TDS2: -2
49 47
TDS1: 1 TDS2: -2
49 49
TDS1: 1 TDS2: -2
49 51
TDS1: 1 TDS2: -2
49 53
TDS1: 1 TDS2: -2
49 55
TDS1: 1 TDS2: -2
49 57
TDS1: 1 TDS2: -2
50 0
TDS1: 1 TDS2: -2
50 2
TDS1: 1 TDS2: -2
50 4
TDS1: 1 TDS2: -2
50 6
TDS1: 1 TDS2: -2
50 8
TDS1: 1 TDS2: -2
50 10
TDS1: 1 TDS2: -2
50 12
TDS1: 1 TDS2: -2
50 14
TDS1: 1 TDS2: -2
50 16
TDS1: 1 TDS2: -2
50 18
TDS1: 1 TDS2: -2
50 20
TDS1: 1 TDS2: -2
50 22
TDS1: 1 TDS2: -2
50 24
TDS1: 1 TDS2: -2
50 26
TDS1: 1 TDS2: -2
50 28
TDS1: 1 TDS2: -2
50 30
TDS1: 1 TDS2: -2
50 32
TDS1: 1 TDS2: -2
50 34
TDS1: 1 TDS2: -2
50 36
TDS1: 1 TDS2: -2
50 38
TDS1: 1 TDS2: -2
50 41
TDS1: 1 TDS2: -2
50 43
TDS1: 1 TDS2: -2
50 45
TDS1: 1 TD

...

181 20
TDS1: 1 TDS2: -2
181 22
TDS1: 1 TDS2: -2
181 24
TDS1: 1 TDS2: -2
181 26
TDS1: 1 TDS2: -2
181 28
TDS1: 1 TDS2: -2
181 30
TDS1: 1 TDS2: -2
181 32
TDS1: 1 TDS2: -2
181 34
TDS1: 1 TDS2: -2
181 36
TDS1: 1 TDS2: -2
181 39
TDS1: 1 TDS2: -2
181 41
TDS1: 1 TDS2: -2
181 43
TDS1: 1 TDS2: -2
181 45
TDS1: 1 TDS2: -2
181 47
TDS1: 1 TDS2: -2
181 49
TDS1: 1 TDS2: -2
181 51
TDS1: 1 TDS2: -2
181 53
TDS1: 1 TDS2: -2
181 55
TDS1: 1 TDS2: -2
181 57
TDS1: 1 TDS2: -2
181 59
TDS1: 1 TDS2: -2
182 1
TDS1: 1 TDS2: -2
182 3
TDS1: 1 TDS2: -2
182 5
TDS1: 1 TDS2: -2
182 7
TDS1: 1 TDS2: -2
182 10
TDS1: 1 TDS2: -2
182 12
TDS1: 1 TDS2: -2
182 14
TDS1: 1 TDS2: -2
182 16
TDS1: 1 TDS2: -2
182 18
TDS1: 1 TDS2: -2
182 20
TDS1: 1 TDS2: -2
182 22
TDS1: 1 TDS2: -2
182 24
TDS1: 1 TDS2: -2
182 26
TDS1: 1 TDS2: -2
182 28
TDS1: 1 TDS2: -2
182 30
TDS1: 1 TDS2: -2
182 32
TDS1: 1 TDS2: -2
182 34
TDS1: 1 TDS2: -2
182 36
TDS1: 1 TDS2: -2
182 38
TDS1: 1 TDS2: -2
182 40
TDS1: 1 TDS2: -2
182 43
TDS1: 1 TDS2: -2
182 45
TDS1: 1 TDS2:

193 12
TDS1: 1 TDS2: -2
193 14
TDS1: 1 TDS2: -2
193 16
TDS1: 1 TDS2: -2
193 18
TDS1: 1 TDS2: -2
193 20
TDS1: 1 TDS2: -2
193 22
TDS1: 1 TDS2: -2
193 24
TDS1: 1 TDS2: -2
193 26
TDS1: 1 TDS2: -2
193 28
TDS1: 1 TDS2: -2
193 30
TDS1: 1 TDS2: -2
193 32
TDS1: 1 TDS2: -2
193 35
TDS1: 1 TDS2: -2
193 37
TDS1: 1 TDS2: -2
193 39
TDS1: 1 TDS2: -2
193 41
TDS1: 1 TDS2: -2
193 43
TDS1: 1 TDS2: -2
193 45
TDS1: 1 TDS2: -2
193 47
TDS1: 1 TDS2: -2
193 49
TDS1: 1 TDS2: -2
193 51
TDS1: 1 TDS2: -2
193 53
TDS1: 1 TDS2: -2
193 55
TDS1: 1 TDS2: -2
193 57
TDS1: 1 TDS2: -2
193 59
TDS1: 1 TDS2: -2
194 1
TDS1: 1 TDS2: -2
194 3
TDS1: 1 TDS2: -2
194 6
TDS1: 1 TDS2: -2
194 8
TDS1: 1 TDS2: -2
194 10
TDS1: 1 TDS2: -2
194 12
TDS1: 1 TDS2: -2
194 14
TDS1: 1 TDS2: -2
194 16
TDS1: 1 TDS2: -2
194 18
TDS1: 1 TDS2: -2
194 20
TDS1: 1 TDS2: -2
194 22
TDS1: 1 TDS2: -2
194 24
TDS1: 1 TDS2: -2
194 26
TDS1: 1 TDS2: -2
194 28
TDS1: 1 TDS2: -2
194 30
TDS1: 1 TDS2: -2
194 32
TDS1: 1 TDS2: -2
194 34
TDS1: 1 TDS2: -2
194 36
TDS1: 1 TDS2:

205 3
TDS1: 1 TDS2: -2
205 5
TDS1: 1 TDS2: -2
205 7
TDS1: 1 TDS2: -2
205 9
TDS1: 1 TDS2: -2
205 11
TDS1: 1 TDS2: -2
205 13
TDS1: 1 TDS2: -2
205 15
TDS1: 1 TDS2: -2
205 17
TDS1: 1 TDS2: -2
205 19
TDS1: 1 TDS2: -2
205 21
TDS1: 1 TDS2: -2
205 23
TDS1: 1 TDS2: -2
205 25
TDS1: 1 TDS2: -2
205 27
TDS1: 1 TDS2: -2
205 30
TDS1: 1 TDS2: -2
205 32
TDS1: 1 TDS2: -2
205 34
TDS1: 1 TDS2: -2
205 36
TDS1: 1 TDS2: -2
205 38
TDS1: 1 TDS2: -2
205 40
TDS1: 1 TDS2: -2
205 42
TDS1: 1 TDS2: -2
205 44
TDS1: 1 TDS2: -2
205 46
TDS1: 1 TDS2: -2
205 48
TDS1: 1 TDS2: -2
205 50
TDS1: 1 TDS2: -2
205 52
TDS1: 1 TDS2: -2
205 54
TDS1: 1 TDS2: -2
205 56
TDS1: 1 TDS2: -2
205 58
TDS1: 1 TDS2: -2
206 1
TDS1: 1 TDS2: -2
206 3
TDS1: 1 TDS2: -2
206 5
TDS1: 1 TDS2: -2
206 7
TDS1: 1 TDS2: -2
206 9
TDS1: 1 TDS2: -2
206 11
TDS1: 1 TDS2: -2
206 13
TDS1: 1 TDS2: -2
206 15
TDS1: 1 TDS2: -2
206 17
TDS1: 1 TDS2: -2
206 19
TDS1: 1 TDS2: -2
206 21
TDS1: 1 TDS2: -2
206 23
TDS1: 1 TDS2: -2
206 25
TDS1: 1 TDS2: -2
206 27
TDS1: 1 TDS2: -2
2

216 53
TDS1: 1 TDS2: -2
216 55
TDS1: 1 TDS2: -2
216 57
TDS1: 1 TDS2: -2
216 59
TDS1: 1 TDS2: -2
217 1
TDS1: 1 TDS2: -2
217 3
TDS1: 1 TDS2: -2
217 5
TDS1: 1 TDS2: -2
217 8
TDS1: 1 TDS2: -2
217 10
TDS1: 1 TDS2: -2
217 12
TDS1: 1 TDS2: -2
217 14
TDS1: 1 TDS2: -2
217 16
TDS1: 1 TDS2: -2
217 18
TDS1: 1 TDS2: -2
217 20
TDS1: 1 TDS2: -2
217 22
TDS1: 1 TDS2: -2
217 24
TDS1: 1 TDS2: -2
217 26
TDS1: 1 TDS2: -2
217 28
TDS1: 1 TDS2: -2
217 30
TDS1: 1 TDS2: -2
217 32
TDS1: 1 TDS2: -2
217 34
TDS1: 1 TDS2: -2
217 36
TDS1: 1 TDS2: -2
217 38
TDS1: 1 TDS2: -2
217 40
TDS1: 1 TDS2: -2
217 42
TDS1: 1 TDS2: -2
217 45
TDS1: 1 TDS2: -2
217 47
TDS1: 1 TDS2: -2
217 49
TDS1: 1 TDS2: -2
217 51
TDS1: 1 TDS2: -2
217 53
TDS1: 1 TDS2: -2
217 55
TDS1: 1 TDS2: -2
217 57
TDS1: 1 TDS2: -2
217 59
TDS1: 1 TDS2: -2
218 1
TDS1: 1 TDS2: -2
218 3
TDS1: 1 TDS2: -2
218 5
TDS1: 1 TDS2: -2
218 7
TDS1: 1 TDS2: -2
218 9
TDS1: 1 TDS2: -2
218 11
TDS1: 1 TDS2: -2
218 13
TDS1: 1 TDS2: -2
218 15
TDS1: 1 TDS2: -2
218 18
TDS1: 1 TDS2: -2
2

228 43
TDS1: 1 TDS2: -2
228 45
TDS1: 1 TDS2: -2
228 47
TDS1: 1 TDS2: -2
228 49
TDS1: 1 TDS2: -2
228 51
TDS1: 1 TDS2: -2
228 53
TDS1: 1 TDS2: -2
228 55
TDS1: 1 TDS2: -2
228 57
TDS1: 1 TDS2: -2
228 59
TDS1: 1 TDS2: -2
229 1
TDS1: 1 TDS2: -2
229 4
TDS1: 1 TDS2: -2
229 6
TDS1: 1 TDS2: -2
229 8
TDS1: 1 TDS2: -2
229 10
TDS1: 1 TDS2: -2
229 12
TDS1: 1 TDS2: -2
229 14
TDS1: 1 TDS2: -2
229 16
TDS1: 1 TDS2: -2
229 18
TDS1: 1 TDS2: -2
229 20
TDS1: 1 TDS2: -2
229 22
TDS1: 1 TDS2: -2
229 24
TDS1: 1 TDS2: -2
229 26
TDS1: 1 TDS2: -2
229 28
TDS1: 1 TDS2: -2
229 30
TDS1: 1 TDS2: -2
229 32
TDS1: 1 TDS2: -2
229 35
TDS1: 1 TDS2: -2
229 37
TDS1: 1 TDS2: -2
229 39
TDS1: 1 TDS2: -2
229 41
TDS1: 1 TDS2: -2
229 43
TDS1: 1 TDS2: -2
229 45
TDS1: 1 TDS2: -2
229 47
TDS1: 1 TDS2: -2
229 49
TDS1: 1 TDS2: -2
229 51
TDS1: 1 TDS2: -2
229 53
TDS1: 1 TDS2: -2
229 55
TDS1: 1 TDS2: -2
229 57
TDS1: 1 TDS2: -2
229 59
TDS1: 1 TDS2: -2
230 1
TDS1: 1 TDS2: -2
230 3
TDS1: 1 TDS2: -2
230 5
TDS1: 1 TDS2: -2
230 7
TDS1: 1 TDS2: -2
